Before...
=========

We developed a model for digits classification in `MLOps/data_science/working/data_science_digits_model.ipynb`.

We migrated to the cloud by adapting the notebook for **Azure Machine Learning** in `MLOps/data_science/cloud_AML_digits_model.ipynb`.

We downloaded the artifact from the Azure ML training job output and saved it in this repository under the name `model`.

In [2]:
import os

data_science_dir = os.path.join('../', "data_science")
aml_job_output = "model"

Now
===

The problem we face
-------------------

To deploy our model on a Azure ML **Real-time Endpoint**, we needed a virtual machine big enough to contain the deep neural network. Not a *very small* but a *small* VM would have been enough, such as Standard_F4s_v2 or Standard_DS2_v2.
However, we did not have enough **Endpoint quota** in our *Azure for Students* subscription for deploying the model on such a VM.

Since we cannot deploy our model on the cloud, we simulate a production environment in this notebook.

The artifact from the AzureML training job output...
----------------------------------------------------

It contains all the necessary data for reproductibility:
- environment and requirements files
- details about the ML framework (scikit-learn, tensorflow, pytorch...)
- serialized model

In [2]:
import os

def walkAmlJobOutput():
    for path, currentDirectory, files in os.walk(aml_job_output):
        for file in files:
            filepath = os.path.join(path, file)
            if os.sep+'.' not in filepath: #hide hidden files
                yield file, filepath

conda_file = None
for file, filepath in walkAmlJobOutput():
    print(filepath, end='')
    if '.yaml' in file:
        if file == "conda.yaml": conda_file = filepath
        print(" <- ENVIRONMENT FILE", end='')
    if file == "saved_model.pb":
        print(" <- SERIALIZED MODEL", end='')
    print()

model\conda.yaml <- ENVIRONMENT FILE
model\MLmodel
model\python_env.yaml <- ENVIRONMENT FILE
model\requirements.txt
model\_summary.txt
model\data\keras_module.txt
model\data\save_format.txt
model\data\model\keras_metadata.pb
model\data\model\saved_model.pb <- SERIALIZED MODEL
model\data\model\variables\variables.data-00000-of-00001
model\data\model\variables\variables.index


In [3]:
print(f"Path to Conda environment file: {conda_file}")

Path to Conda environment file: model\conda.yaml


## Install the right environment to run the model

In [4]:
with open(conda_file, 'r') as f:
    print(f.read())

channels:
- conda-forge
dependencies:
- python=3.8.15
- pip<=21.2.4
- pip:
  - mlflow
  - cffi==1.15.1
  - keras==2.6.0
  - pillow==9.4.0
  - scipy==1.7.1
  - tensorflow==2.6.0
name: mlflow-env



- Open your terminal
- Install the conda environment using `conda env create --name mlops-model-env --file <path/to/conda_file>`
- Activate it with `conda activate mlops-model-env`
- Install your favorite package for using notebooks
- Reopen this notebook

Additionally, install opencv using `conda install -c conda-forge opencv` for the deployement.

Production simulation setup
---------------------------

In [1]:
import os

prod_dir = "./sample_requests"
os.makedirs(prod_dir, exist_ok=True)

In [3]:
from working.dummy_server import DummyServer
import pandas as pd

original_data_quantity = 50

mnist_data = pd.read_csv(os.path.join(data_science_dir, "input", "test.csv"))
mnist_data = mnist_data.tail(original_data_quantity)

prod_data = mnist_data

server = DummyServer(prod_data, prod_dir)

In [4]:
server.setup()

## Deploy the MLFlow model

In [5]:
import os

prod_src_dir = "./working"
os.makedirs(prod_src_dir, exist_ok=True)

In [9]:
%%writefile {prod_src_dir}/score.py
import os, json, random, mlflow
import pandas as pd
import numpy as np
from PIL import Image
from io import StringIO
from mlflow.pyfunc.scoring_server import predictions_to_json


# Set up MLflow tracking
experiment_name = "inference"+str(random.randint(10000,100000))
mlflow.set_experiment(experiment_name)
client = mlflow.MlflowClient()


def init(model_path):
    global model
    global input_schema
    # "model" is the path of the mlflow artifacts when the model was registered. For automl
    # models, this is generally "mlflow-model".
    model = mlflow.pyfunc.load_model(model_path)
    input_schema = model.metadata.get_input_schema()
    os.environ['MLFLOW_TRACKING_FORCE_NO_GIT'] = '1'
    os.environ['GIT_PYTHON_REFRESH'] = '0'

    
    
def parse_json_input(json_data):
    json_df = pd.read_json(json.dumps(json_data['dataframe_split']), orient='split')
    data = json_df.values.astype('float32')
    data = data.reshape(1, 28, 28, 1)
    return data


def average(lst):
    return sum(lst) / len(lst)


def run(raw_data):
    json_data = json.loads(raw_data)
    if 'dataframe_split' not in json_data.keys():
        raise Exception("Request must contain a top level key named 'dataframe_split'")

    data = parse_json_input(json_data)
    
    # Log the data as an artifact
    with mlflow.start_run() as run:
        run_id = run.info.run_id
        Image.fromarray(data.reshape(28,28).astype(np.uint8)).save('data.png')
        mlflow.log_artifact("data.png")
    
    # Make predictions and log them
    with mlflow.start_run(run_id=run_id):
        predictions = model.predict(data)
        
        best_prediction = int(np.argmax(predictions, axis=1))
        best_proba = np.max(predictions, axis=1)
        worst_proba = np.min(predictions, axis=1)
        
        # log predicted probabilities as metrics
        for i, p in enumerate(predictions[0]):
            mlflow.log_metric(f'probability_class_{i}', p)
        
        # Log the predictions as an artifact
        with open("best_pred.txt", "w") as f:
            f.write(str(best_prediction))
        mlflow.log_artifact("best_pred.txt")

        # Log any anomalous predictions as a metric
        if best_proba < 0.9:
            mlflow.log_metric("anomalous_pred_proba", best_proba, step=i)
        if worst_proba > 0.1:
            mlflow.log_metric("anomalous_pred_proba", worst_proba, step=i)
        avg_preds = average(predictions[0])
        if round(avg_preds, 1) != 0.1:
            mlflow.log_metric('anomalous_avg_probas', avg_preds)
        sum_preds = sum(predictions[0])
        if round(sum_preds) != 1:
            mlflow.log_metric('anomalous_sum_probas', sum_preds)
        
    result = StringIO()
    predictions_to_json(best_prediction, result)
    return result.getvalue(), data

Overwriting ./working/score.py


In [7]:
import working.score as score

score.init(aml_job_output)

print(score.model)

2023/03/27 11:00:57 INFO mlflow.tracking.fluent: Experiment with name 'inference43725' does not exist. Creating a new experiment.
C:\Users\boiss\anaconda3\envs\mlops-model-env\lib\site-packages\keras\backend.py:401: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.keras
  run_id: khaki_parsnip_0nh4r3mgw2



**If you had an error loading the serialized model, it is probably due to Azure using another version of Python... See `python_env.yaml`.**

Try downgrading protobuf using `pip install protobuf==3.20.*` in a terminal and restarting the kernel of this notebook.

In [8]:
predictions_data = []
for raw_data in server.do_GET():
    predictions_data.append(score.run(raw_data))

2023/03/27 11:01:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


MlflowException: Unknown model type: <class 'mlflow.pyfunc.PyFuncModel'>

Analysis
--------

Now, you can visualise the output below, but you can also run `mlflow ui` in your terminal and analyse the logged metrics on the local website!

In [ ]:
%matplotlib inline

from working.visualisation import plot

plot(predictions_data, last=15, hspace=0.3, fig_size=(16, 10))